## Starter Code for CSE5243 - Introduction to Data Mining Assignment 3 and 4

In case you encountered any issue in Assignment 2 and the feature matrix is not usable for Assignemnt 3 and 4, you could build on this starter code in order to get meaningful results for Assignment 3 and 4. 

Note that the preprocessing done in this starter code is minimal, just basic tokenization and vocabulary truncation. In order to get good classification and clustering results, it's a good idea to employ more preprocessing techniques, such as [stemming and lemmatization](https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html), feature selection (dimensionality reduction), and data transformation (e.g., normalization). 

This requires NLTK and Python 3 (tested with Python 3.8). If you are using Python 2.7 (deprecated), you will likely need to adapt it accordingly, e.g., the f-string formatting in `print`s and some list comprehension syntax.

PLEASE KEEP THIS CODE CONFIDENFIAL TO 

In [ ]:
import os
import sys
import operator
import time
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')  # needed by word_tokenize

## Step 1: Vocabulary Construction

Scan the dataset to find all the unique tokens (using NLTK's `word_tokenize` for tokenization). Sort the tokens by frequency and only keep the top 40K most frequent tokens in the vocabulary (there are over 200K unique tokens in total using the current tokenization)

In [ ]:
# Assuming this file is put under the same parent directoray as the data directory, and the data directory is named "20news-train"
root_path = "./20news-train"
# The maximum size of the final vocabulary. It's a hyper-parameter. You can change it to see what value gives the best performance.
MAX_VOCAB_SIZE = 40000

start_time = time.time()
vocab_full = {}
n_doc = 0
# Only keep the data dictionaries and ignore possible system files like .DS_Store
folders = [os.path.join(root_path, name) for name in os.listdir(root_path) if os.path.isdir(os.path.join(root_path, name))]
for folder in folders:
    for filename in os.listdir(folder):
        file = os.path.join(folder, filename)
        n_doc += 1
        with open(file, 'r', encoding='utf8', errors='ignore') as f:
            for line in f:
                tokens = word_tokenize(line)
                for token in tokens:
                    vocab_full[token] = vocab_full.get(token, 0) + 1
print(f'{n_doc} documents in total with a total vocab size of {len(vocab_full)}')
vocab_sorted = sorted(vocab_full.items(), key=operator.itemgetter(1), reverse=True)
vocab_truncated = vocab_sorted[:MAX_VOCAB_SIZE]
# Save the vocabulary to file for visual inspection and possible analysis
with open('vocab.txt', 'w') as f:
    for vocab, freq in vocab_truncated:
        f.write(f'{vocab}\t{freq}\n')
# The final vocabulary is a dict mapping each token to its id. frequency information is not needed anymore.
vocab = dict([(token, id) for id, (token, _) in enumerate(vocab_truncated)])
# Since we have truncated the vocabulary, we will encounter many tokens that are not in the vocabulary. We will map all of them to the same 'UNK' token (a common practice in text processing), so we append it to the end of the vocabulary.
vocab['UNK'] = MAX_VOCAB_SIZE
vocab_size = len(vocab)
unk_id = MAX_VOCAB_SIZE
elapsed_time = time.time() - start_time
print(f'Vocabulary construction took {elapsed_time} seconds')

## Step 2: Feature Extraction

Scan the dataset one more time to extract the feature vector and class label of each document.
Note that it's possible to scan the entire dataset only once to both construct the vocabulary and extract the feature vectors. Because we also truncate the vocabulary, we choose to do two scans to make the code mode clear at the cost of runtime efficiency. If efficiency is an issue, you may optimize it by only doing one scan over the dataset.

In [ ]:
# Since we have truncated the vocabulary, it's now reasonable to hold the entire feature matrix in memory (it takes about 3.6GB on a 64-bit machine). If memory is an issue, you could make the vocabulary even smaller or use sparse matrix.
start_time = time.time()
features = np.zeros((n_doc, vocab_size), dtype=int)
print(f'The feature matrix takes {sys.getsizeof(features)} Bytes.')
# The class label of each document
labels = np.zeros(n_doc, dtype=int)
# The mapping from the name of each class label (i.e., the subdictionary name corresponding to a topic) to an integer ID
label2id = {}
label_id = 0
doc_id = 0
for folder in folders:
    label2id[folder] = label_id
    for filename in os.listdir(folder):
        labels[doc_id] = label_id
        file = os.path.join(folder, filename)
        with open(file, 'r', encoding='utf8', errors='ignore') as f:
            for line in f:
                tokens = word_tokenize(line)
                for token in tokens:
                    # if the current token is in the vocabulary, get its ID; otherwise, get the ID of the UNK token
                    token_id = vocab.get(token, unk_id)
                    features[doc_id, token_id] += 1
        doc_id += 1
    label_id += 1
elapsed_time = time.time() - start_time
print(f'Feature extraction took {elapsed_time} seconds')